# Example of geometry optimization
## Batch of two molecules

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Geometry_Optimization_SD
from seqm.seqm_functions.read_xyz import read_xyz
from seqm.seqm_functions.save_xyz import save_xyz
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time

species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,1,1,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [ 0.00,    0.00,    0.00],
                               [ 0.96,    0.00,    0.00],
                               [-0.45,   -0.57,    0.67],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6', # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [1,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.008, force_tol=1.0e-2, max_evl=40).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)


/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/diag_d.py:109: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


1      1.219416e+00 ||-8.825343e-01 -8.825343e-01 ||-8.825343e-01 -8.825343e-01 ||-2.264059e+00 -2.264059e+00 
2      1.014669e+00 ||-8.900513e-01 -7.516916e-03 ||-8.900513e-01 -7.516916e-03 ||-2.289391e+00 -2.533232e-02 
3      8.510361e-01 ||-8.931340e-01 -3.082785e-03 ||-8.931340e-01 -3.082785e-03 ||-2.306793e+00 -1.740112e-02 
4      7.184184e-01 ||-8.947297e-01 -1.595697e-03 ||-8.947297e-01 -1.595697e-03 ||-2.319093e+00 -1.230093e-02 
5      6.097146e-01 ||-8.955843e-01 -8.545348e-04 ||-8.955843e-01 -8.545348e-04 ||-2.327960e+00 -8.866686e-03 
6      5.198888e-01 ||-8.960444e-01 -4.601459e-04 ||-8.960444e-01 -4.601459e-04 ||-2.334447e+00 -6.486379e-03 
7      4.451352e-01 ||-8.962926e-01 -2.481912e-04 ||-8.962926e-01 -2.481912e-04 ||-2.339251e+00 -4.804734e-03 
8      3.825704e-01 ||-8.964269e-01 -1.342557e-04 ||-8.964269e-01 -1.342557e-04 ||-2.342850e+00 -3.598862e-03 
9      3.299449e-01 ||-8.965000e-01 -7.309691e-05 ||-8.965000e-01 -7.309691e-05 ||-2.345573e+00 -2.722642e-03 
1

### Final forces and optimized geometry

In [3]:
molecule.force

tensor([[[ 7.3789e-04,  2.3870e-15, -1.7034e-14],
         [ 7.6861e-04, -4.3779e-14,  1.3083e-14],
         [-7.5325e-04,  1.2152e-03,  4.0286e-16],
         [-7.5325e-04, -1.2152e-03,  3.5482e-15]],

        [[ 7.3789e-04,  2.3870e-15, -1.7034e-14],
         [ 7.6861e-04, -4.3779e-14,  1.3083e-14],
         [-7.5325e-04,  1.2152e-03,  4.0286e-16],
         [-7.5325e-04, -1.2152e-03,  3.5482e-15]],

        [[-5.5423e-03,  5.8053e-03, -6.8237e-03],
         [-3.8182e-03, -5.5440e-03,  6.5167e-03],
         [ 9.3605e-03, -2.6124e-04,  3.0707e-04],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]], device='cuda:0')

In [4]:
molecule.coordinates

tensor([[[ 1.5829e-02,  5.5156e-16, -2.2297e-16],
         [ 1.2266e+00, -6.0692e-16,  3.9896e-16],
         [ 1.8088e+00,  9.3016e-01, -5.4734e-16],
         [ 1.8088e+00, -9.3016e-01,  3.7135e-16]],

        [[ 1.5829e-02,  5.5156e-16, -2.2297e-16],
         [ 1.2266e+00, -6.0692e-16,  3.9896e-16],
         [ 1.8088e+00,  9.3016e-01, -5.4734e-16],
         [ 1.8088e+00, -9.3016e-01,  3.7135e-16]],

        [[-2.6254e-02,  1.5565e-02, -1.8296e-02],
         [ 9.1955e-01, -3.1656e-02,  3.7210e-02],
         [-3.8329e-01, -5.5391e-01,  6.5109e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]], device='cuda:0',
       requires_grad=True)

### Save optimized geometries to .xyz files without final forces.

In [5]:
save_xyz(molecule, 'XYZ', Forces=False)

## Reading starting geometries from .xyz
### Only molecules of the same length are supported for batched xyz_reader.

In [9]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.007, force_tol=1.0e-2, max_evl=60).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.241720e-01 ||4.242794e+00 4.242794e+00 ||4.242794e+00 4.242794e+00 
2      2.836076e-01 ||4.190713e+00 -5.208102e-02 ||4.190713e+00 -5.208102e-02 
3      1.641281e-01 ||4.181992e+00 -8.720535e-03 ||4.181992e+00 -8.720532e-03 
4      1.098067e-01 ||4.178762e+00 -3.229772e-03 ||4.178762e+00 -3.229775e-03 
5      7.981583e-02 ||4.177235e+00 -1.527291e-03 ||4.177235e+00 -1.527290e-03 
6      6.580419e-02 ||4.176424e+00 -8.111486e-04 ||4.176424e+00 -8.111624e-04 
7      5.633106e-02 ||4.175942e+00 -4.824328e-04 ||4.175942e+00 -4.824390e-04 
8      5.242781e-02 ||4.175611e+00 -3.307275e-04 ||4.175611e+00 -3.307071e-04 
9      4.830436e-02 ||4.175357e+00 -2.537246e-04 ||4.175357e+00 -2.537422e-04 
10      4.553573e-02 ||4.175151e+00 -2.064744e-04 ||4.175151e+00 -2.064600e-04 
11      4.362772e-02 ||4.174967e+00 -1.836379e-04 ||4.174967e+00 -1.836198e-04 
12      4.173173e-02 ||4.174802e+00 -1.647795e-04 ||4.174802e+00 -1.647987e-04 
13     

### Alternatively, use zero-padding in .xyz files for molecules of different lengths.

In [10]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'benzene_zero_pad.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.007, force_tol=1.0e-2, max_evl=60).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.241720e-01 ||4.242794e+00 4.242794e+00 ||9.740017e-01 9.740017e-01 
2      2.836076e-01 ||4.190713e+00 -5.208102e-02 ||9.609449e-01 -1.305688e-02 
3      1.721170e-01 ||4.181992e+00 -8.720531e-03 ||9.589003e-01 -2.044565e-03 
4      1.372997e-01 ||4.178762e+00 -3.229780e-03 ||9.576902e-01 -1.210146e-03 
5      1.150457e-01 ||4.177235e+00 -1.527303e-03 ||9.568571e-01 -8.330147e-04 
6      9.455538e-02 ||4.176424e+00 -8.111480e-04 ||9.562795e-01 -5.776414e-04 
7      7.870863e-02 ||4.175942e+00 -4.824371e-04 ||9.558831e-01 -3.964276e-04 
8      6.568225e-02 ||4.175611e+00 -3.307098e-04 ||9.556053e-01 -2.777277e-04 
9      5.474883e-02 ||4.175357e+00 -2.537479e-04 ||9.554117e-01 -1.936680e-04 
10      4.557411e-02 ||4.175151e+00 -2.064323e-04 ||9.552771e-01 -1.346051e-04 
11      4.362692e-02 ||4.174967e+00 -1.836520e-04 ||9.551830e-01 -9.410634e-05 
12      4.173192e-02 ||4.174802e+00 -1.647843e-04 ||9.551176e-01 -6.534846e-05 
13     